# Professor Adams - Financial Risk Management (FRE 6123) 
# Homework #3

### README
This notebook contains solutions for NYU FRE 6123: Financial Risk Management, Homework #3.

- Covers interest rate swaps, forward contracts, Black-Scholes option pricing, and replication strategies.
- Includes both Python code and detailed markdown explanations with formulas.
- All calculations use standard financial mathematics conventions.
- To run: make sure you have Python, pandas, and numpy installed.

---
---
### Part 1 Set up Environment

In [10]:
import math
import pandas as pd
import numpy as np
import math
from dataclasses import dataclass
from typing import List, Tuple

---
---
### Part 2 Answer

#### Question 1 

In [11]:
S_fwd = 1.53          # S (AUD/USD)
F_obs = 1.5075        # Observed forward (AUD/USD)
r_aud = 0.0365        # foreign rate (AUD), continuous
r_usd = 0.0415        # domestic rate (USD), continuous
T     = 3.0           # years
USD0  = 100_000_000   # notional USD

# CIP fair forward under f/d quote (AUD per USD)
F_cip = S_fwd * math.exp((r_aud - r_usd) * T)

# Arbitrage sizing to hold +USD100m at inception:
AUD_borrow = USD0 * S_fwd                 # borrow AUD, convert to USD
USD_T      = USD0 * math.exp(r_usd * T)   # USD investment to T
AUD_repay  = AUD_borrow * math.exp(r_aud * T)
AUD_from_fwd = USD_T * F_obs              # convert USD->AUD via forward at T

# Profits
profit_AUD = AUD_from_fwd - AUD_repay
profit_USD = USD_T - (AUD_repay / F_obs)  # lock USD via same forward
pv_profit  = profit_USD * math.exp(-r_usd * T)

out = {
    "F_cip (AUD/USD)": F_cip,
    "F_obs - F_cip": F_obs - F_cip,
    "t=0 AUD borrowed": AUD_borrow,
    "USD at T": USD_T,
    "AUD repay at T": AUD_repay,
    "AUD from forward at T": AUD_from_fwd,
    "Arb profit at T (AUD)": profit_AUD,
    "Arb profit at T (USD)": profit_USD,
}
for k, v in out.items():
    print(f"{k}: {v:,.6f}")


F_cip (AUD/USD): 1.507221
F_obs - F_cip: 0.000279
t=0 AUD borrowed: 153,000,000.000000
USD at T: 113,258,202.046025
AUD repay at T: 170,705,170.853078
AUD from forward at T: 170,736,739.584382
Arb profit at T (AUD): 31,568.731304
Arb profit at T (USD): 20,941.115293


#### Interpretation

* **Quote AUD/USD:** $S_0 = 1.53$
* **Rates (continuous compounding):** $r_{\mathrm{USD}} = 0.0415$, $r_{\mathrm{AUD}} = 0.0365$
* **Maturity:** $T = 3$ years
* **Observed forward:** $F_{0,3} = 1.5075$ (AUD per 1 USD)
* **Base notional:** USD $100{,}000{,}000$

##### 1) CIP fair forward (f/d quote: AUD per USD)

$$
F_{0,T}^{\text{CIP}} = S_0 e^{(r_{\text{AUD}}-r_{\text{USD}})T}
$$

Plug in values:
$$
F_{0,3}^{\text{CIP}} = 1.53 \times e^{(0.0365-0.0415)\cdot 3} \approx \mathbf{1.5072213}\ \text{AUD/USD}
$$

Compare with market: $F_{0,3}^{\text{mkt}} = 1.5075 > 1.5072213$
→ **forward is slightly overpriced**.

##### 2) Arbitrage direction (with f/d quote)

Because $F_{0,3}^{\text{mkt}} > F_{0,3}^{\text{CIP}}$ :
**Borrow AUD → spot-convert to USD → invest in USD → enter a forward to sell USD / buy AUD** (use the overpriced forward at maturity to get more AUD to repay the AUD loan).

##### 3) Cash flows (USD notional = 100 million)

**At $t=0$:**
* Borrow AUD:
  $$
  \text{AUD}_0 = 100{,}000{,}000 \times 1.53 = 153{,}000{,}000\ \text{AUD}
  $$
* Spot convert to USD ($=100{,}000{,}000$); invest at $r_{\text{USD}}$.
* Short USD / long AUD **forward** at $F=1.5075$ (AUD per 1 USD).

**At $t=3$:**
* USD investment:
  $$
  100{,}000{,}000 \times e^{0.0415\cdot 3} = \mathbf{113{,}258{,}202.046}\ \text{USD}
  $$
* Convert USD to AUD via forward:
  $$
  113{,}258{,}202.046 \times 1.5075 = \mathbf{170{,}736{,}739.584}\ \text{AUD}
  $$
* AUD loan repayment:
  $$
  153{,}000{,}000 \times e^{0.0365\cdot 3} = \mathbf{170{,}705{,}170.853}\ \text{AUD}
  $$

##### 4) Arbitrage profit

* **Terminal profit (AUD):**
  $$
  \Pi_T^{\text{AUD}} = 170{,}736{,}739.584 - 170{,}705{,}170.853 = \boxed{31{,}568.731\ \text{AUD}}
  $$
* **Terminal profit (USD):**
  $$
  \Pi_T^{\text{USD}} = 113{,}258{,}202.046 - \frac{170{,}705{,}170.853}{1.5075} = \boxed{20{,}941.115\ \text{USD}}
  $$

##### 5) CIP Function (f/d quote)

$$
\frac{F_{0,T}}{S_0} = e^{(r_{\text{AUD}}-r_{\text{USD}})T}
$$

When the observed forward is **above** the CIP value under f/d quoting, borrowing AUD, investing USD, and **selling USD for AUD forward** locks in a small, risk-free spread.

---
---

#### Question 2

In [12]:
def regn_forward_and_collateral(
    S0=945.0,            # spot price (USD/share)
    r=0.0425,            # risk-free rate (continuous compounding)
    T=0.5,               # years to maturity (6 months)
    n_shares=1000,       # forward contract shares
    collateral_floor=100_000.0,  # minimum collateral (USD)
    t_check=0.25,        # checkpoint time in years (3 months)
    S_t_check=845.0      # spot at checkpoint
):
    """
    Non-dividend stock forward pricing & collateral requirement.
    Uses continuous compounding and price/base convention consistent with CIP-style formulas.

    Returns a dict with the forward price, inception collateral, and
    checkpoint collateral when spot is S_t_check at time t_check.
    """

    # (a) Forward price (non-dividend, continuous compounding)
    F0 = S0 * math.exp(r * T)
    K = F0  # delivery price fixed at inception

    # (b) Forward value to the LONG at time tau: V_tau = S_tau - K * exp(-r*(T - tau))
    # Inception:
    V0 = S0 - K * math.exp(-r * T)  # should be ~0 by construction
    collat_0 = max(collateral_floor, max(0.0, -V0) * n_shares)

    # Checkpoint at t_check with S_t_check:
    Vt = S_t_check - K * math.exp(-r * (T - t_check))
    collat_t = max(collateral_floor, max(0.0, -Vt) * n_shares)

    # Pretty prints
    print("=== (a) Forward Price ===")
    print(f"Spot S0                 : {S0:,.4f} USD/share")
    print(f"Risk-free r (cont.)     : {r:.6f}")
    print(f"Maturity T              : {T:.4f} years")
    print(f"Forward Price F0        : {F0:,.6f} USD/share")
    print(f"Contract Notional       : {F0 * n_shares:,.2f} USD\n")

    print("=== (b) Collateral Requirement ===")
    print(f"Collateral floor         : {collateral_floor:,.2f} USD")
    print(f"Inception V0 (long)      : {V0:,.6f} USD/share")
    print(f"Collateral at inception  : {collat_0:,.2f} USD")
    print(f"Checkpoint t             : {t_check:.4f} years")
    print(f"Spot at t, S_t           : {S_t_check:,.4f} USD/share")
    print(f"V_t (long)               : {Vt:,.6f} USD/share")
    print(f"Collateral at t          : {collat_t:,.2f} USD")

    return {
        "F0_per_share": F0,
        "contract_notional": F0 * n_shares,
        "V0_per_share": V0,
        "collateral_inception": collat_0,
        "t_check": t_check,
        "S_t_check": S_t_check,
        "Vt_per_share": Vt,
        "collateral_at_t": collat_t,
    }

# --- Run with the problem's inputs ---
result = regn_forward_and_collateral()

=== (a) Forward Price ===
Spot S0                 : 945.0000 USD/share
Risk-free r (cont.)     : 0.042500
Maturity T              : 0.5000 years
Forward Price F0        : 965.296133 USD/share
Contract Notional       : 965,296.13 USD

=== (b) Collateral Requirement ===
Collateral floor         : 100,000.00 USD
Inception V0 (long)      : 0.000000 USD/share
Collateral at inception  : 100,000.00 USD
Checkpoint t             : 0.2500 years
Spot at t, S_t           : 845.0000 USD/share
V_t (long)               : -110.094155 USD/share
Collateral at t          : 110,094.16 USD


#### Interpretation

##### **Answer (a) Forward Price**

$$
F_0 = S_0 e^{rT}
= 945 \times e^{0.0425\times 0.5}
\approx \boxed{965.2961\ \text{USD per share}}
$$

Contract notional for $(1{,}000)$ shares: $(\approx \$965{,}296.13)$.


##### **Answer (b) Collateral Requirement**

**Rule:** Collateral the manager must post each day is the greater of **USD 100,000** or **100% of the mark-to-market loss** on the forward (i.e., the negative value of the long forward).

* **Forward value to the long at time \(t\)** (non-dividend, continuous compounding):
  $$
  V_t = S_t - K e^{-r (T-t)},\qquad K=F_0.
  $$

**At inception \((t=0)\)**  
$(V_0 = 0 \Rightarrow)$ collateral $(=\max(100{,}000,\ 0)=\boxed{\$100{,}000}$\).

**At 3 months \(t=0.25, S_t=845\)**
$$
V_{0.25}
= 845 - 965.2961 \times e^{-0.0425\,(0.5-0.25)}
\approx -110.0942\ \text{USD per share}.
$$

Total MTM loss $(= 110.0942 \times 1{,}000 \approx \$110{,}094.16)$.  
Collateral $max(100{,}000,110{,}094.16)$ $=>{\$110{,}094.16}$.




---
---
#### Question 3

In [13]:
# parameters
BOND_MOD_DURS = [1.873, 4.329, 7.538]   # (2y, 5y, 10y)
BOND_CONVEX   = [3.6, 19.5, 58.5]       # not directly needed here
BOND_COUPONS  = [0.045, 0.050, 0.055]   # annual coupons; not needed since ModDur given
MV_EACH       = 100_000_000.0           # USD 100m each
N_BONDS       = 3
MV_BONDS_TOT  = MV_EACH * N_BONDS       # 300m

# Swap: pay-fixed, receive-float
SWAP_NOTIONAL = 200_000_000.0
SWAP_FIXED_RATE = 0.0515                # 5.15% (semi-annual)
FLOAT_CURR_Y   = 0.0425                 # 6m money market rate 4.25%
SWAP_TENOR_YRS = 5.0
FREQ = 2                                # semi-annual

# -----------------------------
# Helper: fixed-coupon bond mod duration (price=par if y=c)
# -----------------------------
def fixed_bond_mod_duration_par(c_annual: float, y_annual: float, T: float, m: int) -> float:
    """
    Modified duration for a fixed-rate bond assuming it's priced at par: y_annual = c_annual.
    Uses discrete compounding with m periods per year (e.g., m=2 for semi-annual).
    Face = 1. Returns duration in YEARS.
    """
    assert m > 0
    per = int(round(T * m))
    c = c_annual / m
    y = y_annual / m
    face = 1.0

    # Cashflows & discount factors
    times = [(i + 1) / m for i in range(per)]
    cflows = [c * face] * per
    cflows[-1] += face  # add redemption

    disc = [1.0 / ((1.0 + y) ** (i + 1)) for i in range(per)]
    pv = sum(cf * d for cf, d in zip(cflows, disc))
    d_mac = sum(t * cf * d for t, cf, d in zip(times, cflows, disc)) / pv  # Macaulay
    d_mod = d_mac / (1.0 + y)  # Modified
    return d_mod


# (a) Swap modified duration
# D_swap (pay-fixed, receive-float) = D_float - D_fixed   per $ notional
# Float leg duration ≈ time-to-next-reset / (1 + y_float/m)
D_fixed = fixed_bond_mod_duration_par(
    c_annual=SWAP_FIXED_RATE,
    y_annual=SWAP_FIXED_RATE,  # par
    T=SWAP_TENOR_YRS,
    m=FREQ
)

D_float = (0.5) / (1.0 + FLOAT_CURR_Y / FREQ)  # time to next reset = 0.5y
D_swap_pay_fixed = D_float - D_fixed


# (b) Portfolio duration with swap
# Bonds-only duration = equal-value-weighted average of given ModDur
# Swap PV ~ 0 at inception ⇒ denominator is MV of bonds only.
D_bonds_only = sum(BOND_MOD_DURS) / len(BOND_MOD_DURS)          # equal value weights (100m each)
DD_bonds = MV_BONDS_TOT * D_bonds_only
DD_swap  = SWAP_NOTIONAL * D_swap_pay_fixed
D_total_with_swap = (DD_bonds + DD_swap) / MV_BONDS_TOT

# Quick printout
print("=== (a) Swap Modified Duration (pay-fixed, receive-float) ===")
print(f"Fixed-leg ModDur (5y, 5.15%, SA) : {D_fixed:.10f}")
print(f"Float-leg ModDur (6m reset)      : {D_float:.10f}")
print(f"Swap ModDur per $ notional       : {D_swap_pay_fixed:.10f}\n")

print("=== (b) Portfolio Modified Duration ===")
print(f"Bonds-only ModDur (equal value)  : {D_bonds_only:.10f}")
print(f"Portfolio ModDur with swap       : {D_total_with_swap:.10f}\n")

# For quick access programmatically:
results = {
    "D_fixed_leg": D_fixed,
    "D_float_leg": D_float,
    "D_swap_pay_fixed": D_swap_pay_fixed,
    "D_bonds_only": D_bonds_only,
    "D_portfolio_with_swap": D_total_with_swap
}
df_results = pd.DataFrame(list(results.items()), columns=['Parameters', 'Value'])
df_results['Value'] = df_results['Value'].astype(float).round(6)
display(df_results)


=== (a) Swap Modified Duration (pay-fixed, receive-float) ===
Fixed-leg ModDur (5y, 5.15%, SA) : 4.3591213116
Float-leg ModDur (6m reset)      : 0.4895960832
Swap ModDur per $ notional       : -3.8695252284

=== (b) Portfolio Modified Duration ===
Bonds-only ModDur (equal value)  : 4.5800000000
Portfolio ModDur with swap       : 2.0003165144



,Parameters,Value
0,D_fixed_leg,4.359121
1,D_float_leg,0.489596
2,D_swap_pay_fixed,-3.869525
3,D_bonds_only,4.580000
4,D_portfolio_with_swap,2.000317


#### Interpretation


* **(a) Swap duration:**  
  Treat the IRS as **long a floater** and **short a fixed-rate bond**:  
  $$
  D_{\text{swap}} = D_{\text{float}} - D_{\text{fixed}}, \quad D_{\text{float}} \approx \frac{0.5}{1 + y_f/2}.
  $$
  Numerically, $D_{\text{swap}} \approx -3.8695$ per \$ notional.

* **(b) Portfolio duration with swap:**  
  Bonds-only (equal value):  
  $$
  D_{\text{bonds}} = \tfrac{1}{3}(1.873+4.329+7.538) = 4.58.
  $$
  Including a USD 200m pay-fixed swap (PV $\approx 0$ at inception),  
  $$
  D_{\text{total}} = \frac{300\text{m}\cdot 4.58 + 200\text{m}\cdot (-3.8695)}{300\text{m}} \approx 2.0003.
  $$

* **(c) Exposure shift:**  
  The pay-fixed swap adds **negative duration** focused in the **5y belly**, cutting the portfolio’s modified duration from **4.58** to **≈2.00**。The portfolio is now better positioned against **rate increases** (lower DV01) and benefits most from **bearish moves concentrated around 5y** (e.g., bear-steepening in the belly relative to 2y/10y).

---
---
#### Question 4

#### Answers

**Forward on a non-dividend stock (continuous compounding):**
$$
F_0 = S_0 e^{rT}.
$$

**Put–call parity (European, same $K$, $T$):**
$$
C - P = S_0 - K e^{-rT}.
$$

**Synthetic long forward via options:** Take long call $+C(K,T)$ and short put $-P(K,T)$ with the same strike $K$ and maturity $T$.
Payoff at $T$:
$$
(C-P)_T = \max(S_T-K,0) - \max(K-S_T,0) = S_T - K,
$$
which is exactly the forward payoff (long forward, delivery price $K$).

Initial value:
$$
(C-P)_0 = S_0 - K e^{-rT}.
$$
If you choose $K=F_0=S_0 e^{rT}$, then
$$
(C-P)_0 = S_0 - F_0 e^{-rT} = 0,
$$
so the option combo has zero cost at inception—just like a standard forward.

**Equivalent cash–stock replication:** long stock and borrow the present value of the delivery price,
$$
\text{Long } S_0 + \text{Short bond with PV } K e^{-rT} \implies \text{payoff } S_T - K.
$$

Thus there are two exact replications of a long forward:
- **Options:** $+$Call$(K,T) -$Put$(K,T)$.
- **Cash + Stock:** $S_0 - K e^{-rT}$ (long stock financed by borrowing PV of $K$).

---
---
#### Question 5

In [14]:
# ---- Standard normal CDF ----
def N(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

# ---- Core formulas ----
def d1(S: float, K: float, r: float, sigma: float, T: float) -> float:
    return (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S: float, K: float, r: float, sigma: float, T: float) -> float:
    return d1(S, K, r, sigma, T) - sigma * math.sqrt(T)

def call_price(S: float, K: float, r: float, sigma: float, T: float) -> float:
    _d1 = d1(S, K, r, sigma, T)
    _d2 = _d1 - sigma * math.sqrt(T)
    return S * N(_d1) - K * math.exp(-r * T) * N(_d2)

def put_price(S: float, K: float, r: float, sigma: float, T: float) -> float:
    _d1 = d1(S, K, r, sigma, T)
    _d2 = _d1 - sigma * math.sqrt(T)
    return K * math.exp(-r * T) * N(-_d2) - S * N(-_d1)

# Results output
S0, K, r, sigma, T = 100.0, 110.0, 0.10, 0.30, 3.0
result_dict = {
    "d1": d1(S0, K, r, sigma, T),
    "d2": d2(S0, K, r, sigma, T),
    "Call Price": call_price(S0, K, r, sigma, T),
    "Put Price": put_price(S0, K, r, sigma, T)
}
import pandas as pd
df_result = pd.DataFrame(list(result_dict.items()), columns=["Parameter", "Value"])
df_result["Value"] = df_result["Value"].astype(float).round(6)
display(df_result)

,Parameter,Value
0,d1,0.653733
1,d2,0.134118
2,Call Price,29.243700
3,Put Price,10.733704


#### Interpretation

Using the Black–Scholes formula (continuous compounding, no dividends):

$$
d_1 = \frac{\ln(S_0/K) + (r + \frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}, \quad d_2 = d_1 - \sigma\sqrt{T}
$$

European call option price:
$$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$$
European put option price:
$$
P = K e^{-rT} N(-d_2) - S_0 N(-d_1)
$$

Plugging in $S_0=100,\ K=110,\ r=0.10,\ \sigma=0.30,\ T=3$ gives:
$$
d_1 \approx 0.6537, \quad d_2 \approx 0.1341, \quad C \approx 29.2437, \quad P \approx 10.7337
$$

---
---